## General Workflow

The code in this Jupyter notebook is designed to identify both active and inactive creators within an ArcGIS Online organization. The workflow is as follows:

1. Import necessary libraries.
2. Prompt the user to enter their ArcGIS Online credentials.
3. Connect to ArcGIS Online using the provided credentials.
4. Retrieve all users and their user types.
5. Identify users who have the ability to create data (based on their user types).
6. Check if these users have published any items within the past six months, have published before but not within the past six months, or have never published any items.

---

**Cell 1: Importing Libraries**

---

In [1]:
from arcgis.gis import GIS

import time
from datetime import datetime

import getpass

C:\Users\rami8629\AppData\Roaming\Python\Python39\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


---

**Cell 2: User Authentication**

In this cell, we prompt the user to enter their ArcGIS Online credentials. These credentials are used to establish a connection to ArcGIS Online.

---


In [2]:
username = input("Enter your username: ")
password = getpass.getpass("Enter your password: ")

# Connect to ArcGIS Online
gis = GIS("https://www.arcgis.com", username, password)


---

**Cell 3: Retrieving Users and Filtering Creators**

In this cell, we retrieve all users and their user types. We then identify the users who have the ability to create data based on their user types. The user types that have the ability to create data are 'creatorUT', 'GISProfessionalAdvUT', 'GISProfessionalBasicUT', and 'GISProfessionalStdUT'. We filter out the users who can create data and store their usernames and user types for further processing.


---

In [3]:
# Get all users
all_users = gis.users.search()
# Get the usernames and user types
usernames_and_usertypes = [(user.username, user.userLicenseTypeId) for user in all_users]

# User types that can create data
creator_types = ['creatorUT', 'GISProfessionalAdvUT', 'GISProfessionalBasicUT', 'GISProfessionalStdUT']

# Filter users who can create data
users_can_create_data = [user for user in usernames_and_usertypes if user[1] in creator_types]

---

**Cell 4: Identifying Active, Inactive, and Never Published Users**

In this cell, we identify active, inactive, and never published users among those who can create data. We first get the current timestamp and the timestamp for six months ago. We then initialize three lists: one for users who added items in the past six months (active users), one for users who didn't add items in the past six months but have published before (inactive users), and one for users who have never published any items.

We iterate over the users who can create data. For each user, we get their folders (including the root folder) and the items in each folder. We check if any item was created in the past six months. If so, the user is considered active and is added to the list of users who added items. If not, we check if the user has ever published any items. If they have, they are considered inactive and are added to the list of users who didn't add items. If they have never published any items, they are added to the list of users who never published.

Finally, we print the usernames and user types of active, inactive, and never published users.

<div style="color: red;">Disclaimer: This piece of code may take a couple of minutes to execute, depending on the size of the organization and the number of users there.</div>

---

In [4]:
# Get the current timestamp in milliseconds
current_timestamp = int(time.time() * 1000)

# Get the timestamp for 6 months ago
six_months_ago = current_timestamp - (6 * 30 * 24 * 60 * 60 * 1000)

# Initialize lists for users who added items in the past 6 months, those who didn't, and those who never published
users_added_items = []
users_didnt_add_items = []
users_never_published = []

# Iterate over the users who can create data
for username, usertype in users_can_create_data:
    # Get the user
    user = gis.users.get(username)
    
    # Get the user's folders
    folders = user.folders
    folders.append(None)  # Add None to check the root folder
    
    # Initialize a flag to indicate whether the user has published any items in the past 6 months
    has_published_recently = False
    # Initialize a flag to indicate whether the user has ever published any items
    has_ever_published = False
    
    # Iterate over the folders
    for folder in folders:
        # Get the items in the folder
        items = user.items(folder=folder, max_items=100)
        
        # Check if any item was created in the past 6 months
        if any(item['created'] > six_months_ago for item in items):
            has_published_recently = True
            has_ever_published = True
            break
        # Check if the user has ever published any items
        elif items:
            has_ever_published = True
    
    # Add the user to the appropriate list
    if has_published_recently:
        users_added_items.append((username, usertype))
    elif has_ever_published:
        users_didnt_add_items.append((username, usertype))
    else:
        users_never_published.append((username, usertype))

# Print usernames and user types of users who added items in the past 6 months, those who didn't, and those who never published
print("Users who added items in the past 6 months:")
for username, usertype in users_added_items:
    print(f"Username: {username}, User Type: {usertype}")

print("\nUsers who didn't add items in the past 6 months but have published before:")
for username, usertype in users_didnt_add_items:
    print(f"Username: {username}, User Type: {usertype}")

print("\nUsers who never published any items:")
for username, usertype in users_never_published:
    print(f"Username: {username}, User Type: {usertype}")

Users who added items in the past 6 months:
Username: akim.aiddev, User Type: creatorUT
Username: apfister_EsriAidDev, User Type: creatorUT
Username: avazquezgbd, User Type: creatorUT
Username: cterborgh_aid, User Type: creatorUT
Username: hdenoord_esriaiddev, User Type: creatorUT
Username: ltroup_EsriAidDev, User Type: creatorUT
Username: ralouta.aiddev, User Type: creatorUT
Username: ssawaya_aid, User Type: creatorUT
Username: vdesrosier_esriaiddev, User Type: GISProfessionalAdvUT
Username: ykandula.esriaiddev, User Type: creatorUT

Users who didn't add items in the past 6 months but have published before:
Username: AidDevService, User Type: creatorUT
Username: ajenkins_EsriAidDev, User Type: creatorUT
Username: amakowicki_aid, User Type: GISProfessionalAdvUT
Username: bmccardle_aid, User Type: GISProfessionalAdvUT
Username: boneill.aid, User Type: creatorUT
Username: bstayer_EsriAidDev, User Type: creatorUT
Username: cbrigham_aid, User Type: creatorUT
Username: ckiefer_EsriAidDev, U

---

**Cell 5: Printing the Number of Active, Inactive, and Never Published Users**

This code snippet prints the number of users in each category: active, inactive, and never published. 

---


In [5]:
# Get the organization name
org_name = gis.properties.name

print(f"In the organization '{org_name}', the number of users who added items in the past 6 months: {len(users_added_items)}")
print(f"In the organization '{org_name}', the number of users who didn't add items in the past 6 months but have published before: {len(users_didnt_add_items)}")
print(f"In the organization '{org_name}', the number of users who never published any items: {len(users_never_published)}")

In the organization 'Esri Aid & Development Team', the number of users who added items in the past 6 months: 10
In the organization 'Esri Aid & Development Team', the number of users who didn't add items in the past 6 months but have published before: 30
In the organization 'Esri Aid & Development Team', the number of users who never published any items: 42
